In [1]:
%matplotlib
# to add the spiking model codebase to the path
import sys, os
sys.path.append('..')
from datetime import datetime

from results import Results
from analysis import (generate_description, FR_weights, determine_first_convergent_trial, determine_drift_rates,
                      FR_scatter, FR_hist2d, FR_hist1d_compare, FR_weights_std, weights_scatter_matrix,
                      weights_hist1d_compare, weights_matrix_compare, combine_two_sessions, calculate_convergence_index,
                      calculate_net_current_units)
from seed_files import (crossHomeoPreScalar, crossHomeoPreScalarHomeo, crossHomeoPreScalarHomeoRemean, crossHomeoPreOuterHomeo,
                        crossHomeoPreOuter, crossHomeoPreOuterHomeoNoLims, crossHomeoPreOuterHomeoNoUpperLims,
                        crossHomeoPreScalarExplodeSolve, crossHomeoPreScalarHomeoExplodeSolve,
                        crossHomeoPreOuterExplodeSolve, crossHomeoPreOuterHomeoExplodeSolve)
from generate import adjacency_matrix_from_flat_inds
from stats import moving_average

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from brian2 import pA, second, Hz

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 9221 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [2]:
import matplotlib.font_manager as fm

In [3]:
plt.close('all')

In [17]:
basePath = 'C:/Users/mikejseay/Documents/BrianResults/'
movAvgWidth = 31
# analysisDicts = (crossHomeoPreScalar, crossHomeoPreScalarHomeo, crossHomeoPreOuter, crossHomeoPreOuterHomeo)
# analysisLabels = ()
# analysisDicts = (crossHomeoPreOuterHomeo, crossHomeoPreOuterHomeoNoUpperLims)
# analysisLabels = ('cross-homeo-pre-outer-homeo',
#                   'cross-homeo-pre-outer-homeo-noUpperLims',)
analysisDicts = (crossHomeoPreScalarExplodeSolve, crossHomeoPreScalarHomeoExplodeSolve,
                 crossHomeoPreOuterExplodeSolve, crossHomeoPreOuterHomeoExplodeSolve)
analysisLabels = ('cross-homeo-pre-scalar-eS', 'cross-homeo-pre-scalar-homeo-eS',
                  'cross-homeo-pre-outer-eS', 'cross-homeo-pre-outer-homeo-eS', )
# useSeeds = list(range(9))
# useSeeds = [0, 1, 3, 4, 5, 6, 7, 8]
useSeeds = [0, 1, 2, 3, 4, 5, 6, 7, 8]
allSeries = []

nRules = len(analysisDicts)
nSeeds = len(useSeeds)
# nTrials = 13530
nTrials = 6000
nUnitsExc = 1600
nUnitsInh = 400

trialUpFRExcAll = np.empty((nRules, nSeeds, nTrials))
trialUpFRInhAll = np.empty((nRules, nSeeds, nTrials))
trialUpDurAll = np.empty((nRules, nSeeds, nTrials))

# trialUpFRExcUnitsAll = np.empty((nRules, nSeeds, nTrials, nUnitsExc))
# trialUpFRInhUnitsAll = np.empty((nRules, nSeeds, nTrials, nUnitsInh))

trialwEEAll = np.empty((nRules, nSeeds, nTrials))
trialwIEAll = np.empty((nRules, nSeeds, nTrials))
trialwEIAll = np.empty((nRules, nSeeds, nTrials))
trialwIIAll = np.empty((nRules, nSeeds, nTrials))
MSEExcUnitsHatAll = np.empty((nRules, nSeeds, nTrials))
MSEInhUnitsHatAll = np.empty((nRules, nSeeds, nTrials))
MSEExcAll = np.empty((nRules, nSeeds, nTrials))
MSEInhAll = np.empty((nRules, nSeeds, nTrials))
distanceFromSetAll = np.empty((nRules, nSeeds, nTrials))
distanceFromSetUnitsAll = np.empty((nRules, nSeeds, nTrials))
frSumExcAll = np.empty((nRules, nSeeds, nTrials - movAvgWidth + 1))
frSumInhAll = np.empty((nRules, nSeeds, nTrials - movAvgWidth + 1))
wSumAll = np.empty((nRules, nSeeds, nTrials - movAvgWidth))
firstConvergentIndAll = np.empty((nRules, nSeeds))
terminalInd = -2000

for analysisInd, analysisDict in enumerate(analysisDicts):
    targetPath = basePath + analysisDict['subfolder']
    for useSeed in useSeeds:
        if len(analysisDict[useSeed]) == 2:
            targetFile1, targetFile2 = analysisDict[useSeed]
            R1 = Results()
            R1.init_from_file(targetFile1, targetPath)
            R2 = Results()
            R2.init_from_file(targetFile2, targetPath)
            R = combine_two_sessions(R1, R2)
        else:
            targetFile = analysisDict[useSeed]
            R = Results()
            R.init_from_file(targetFile, targetPath)
        firstConvergentInd = determine_first_convergent_trial(R)
        
        # preparatory calculations
        MSEExc = (R.trialUpFRExc - R.p['setUpFRExc'] / Hz) ** 2
        MSEInh = (R.trialUpFRInh - R.p['setUpFRInh'] / Hz) ** 2
        MSEHat = (MSEExc + MSEInh) / 2
        distanceFromSet = np.sqrt((R.trialUpFRExc - R.p['setUpFRExc'] / Hz) ** 2 + (R.trialUpFRInh - R.p['setUpFRInh'] / Hz) ** 2)
        MSEExcUnits = (R.trialUpFRExcUnits - R.p['setUpFRExc'] / Hz) ** 2
        MSEExcUnitsHat = MSEExcUnits.mean(1)
        MSEInhUnits = (R.trialUpFRInhUnits - R.p['setUpFRInh'] / Hz) ** 2
        MSEInhUnitsHat = MSEInhUnits.mean(1)
        MSEUnitsHat = (MSEExcUnitsHat + MSEInhUnitsHat) / 2
        distanceFromSetUnits = np.sqrt(MSEExcUnits.sum(1) + MSEInhUnits.sum(1))
        trialdwEE = np.ediff1d(R.trialwEE)
        trialdwIE = np.ediff1d(R.trialwIE)
        trialdwEI = np.ediff1d(R.trialwEI)
        trialdwII = np.ediff1d(R.trialwII)
        allAbsdW = np.fabs(trialdwEE) + np.fabs(trialdwIE) + np.fabs(trialdwEI) + np.fabs(trialdwII)
        wSum = calculate_convergence_index(R)
        frSum = moving_average(MSEExcUnits.mean(1), movAvgWidth) + moving_average(MSEInhUnits.mean(1), movAvgWidth)
        movAvgWidth=31
        frSumExc = moving_average(MSEExcUnits.mean(1), movAvgWidth)
        frSumInh = moving_average(MSEInhUnits.mean(1), movAvgWidth)
        frSum = frSumExc + frSumInh
        
        # scalar quantifiers
        meanAbsdwEE = np.fabs(trialdwEE).mean()
        meanAbsdwIE = np.fabs(trialdwIE).mean()
        meanAbsdwEI = np.fabs(trialdwEI).mean()
        meanAbsdwII = np.fabs(trialdwII).mean()
        meanAbsdwSum = meanAbsdwEE + meanAbsdwIE + meanAbsdwEI + meanAbsdwII
        meanSumAbsMAvgDiffdW = wSum.mean()
        meanSumAbsMAvgDiffTerminaldW = wSum[terminalInd:].mean()
        meanSumAbsMAvgDiffFRMSE = frSum.mean()
        meanSumAbsMAvgDiffTerminalFRMSE = frSum[terminalInd:].mean()
        meanAbsTerminaldwEE = np.fabs(trialdwEE[terminalInd:]).mean()
        meanAbsTerminaldwIE = np.fabs(trialdwIE[terminalInd:]).mean()
        meanAbsTerminaldwEI = np.fabs(trialdwEI[terminalInd:]).mean()
        meanAbsTerminaldwII = np.fabs(trialdwII[terminalInd:]).mean()
        meanAbsTerminaldwSum = meanAbsTerminaldwEE + meanAbsTerminaldwIE + meanAbsTerminaldwEI + meanAbsTerminaldwII
        
        meanDistanceFromSet = distanceFromSet.mean()
        meanTerminalDistanceFromSet = distanceFromSet[terminalInd:].mean()
        meanDistanceFromSetUnits = distanceFromSetUnits.mean()
        meanTerminalDistanceFromSetUnits = distanceFromSetUnits[terminalInd:].mean()
        
        minMSEExc = MSEExc.min()
        meanMSEExc = MSEExc.mean()
        meanTerminalMSEExc = MSEExc[terminalInd:].mean()
        minMSEExcUnits = MSEExcUnitsHat.min()
        meanMSEExcUnits = MSEExcUnitsHat.mean()
        meanTerminalMSEExcUnits = MSEExcUnitsHat[terminalInd:].mean()
        minMSEInh = MSEInh.min()
        meanMSEInh = MSEInh.mean()
        meanTerminalMSEInh = MSEInh[terminalInd:].mean()
        minMSEInhUnits = MSEInhUnitsHat.min()
        meanMSEInhUnits = MSEInhUnitsHat.mean()
        meanTerminalMSEInhUnits = MSEInhUnitsHat[terminalInd:].mean()
        
        meanTerminalMSE = MSEHat[terminalInd:].mean()
        meanTerminalMSEUnits = MSEUnitsHat[terminalInd:].mean()
        
        meanInitialMSE = MSEHat[0]
        meanInitialMSEUnits = MSEUnitsHat[0]
        
        MSEExcUnitsHatAll[analysisInd, useSeed, :] = MSEExcUnitsHat
        MSEInhUnitsHatAll[analysisInd, useSeed, :] = MSEInhUnitsHat
        MSEExcAll[analysisInd, useSeed, :] = MSEExc
        MSEInhAll[analysisInd, useSeed, :] = MSEInh
        distanceFromSetAll[analysisInd, useSeed, :] = distanceFromSet
        distanceFromSetUnitsAll[analysisInd, useSeed, :] = distanceFromSetUnits
        
        frSumExcAll[analysisInd, useSeed, :] = frSumExc
        frSumInhAll[analysisInd, useSeed, :] = frSumInh
        wSumAll[analysisInd, useSeed, :] = wSum
        
        firstConvergentIndAll[analysisInd, useSeed] = firstConvergentInd
        trialwEEAll[analysisInd, useSeed, :] = R.trialwEE
        trialwIEAll[analysisInd, useSeed, :] = R.trialwIE
        trialwEIAll[analysisInd, useSeed, :] = R.trialwEI
        trialwIIAll[analysisInd, useSeed, :] = R.trialwII
        trialUpFRExcAll[analysisInd, useSeed, :] = R.trialUpFRExc
        trialUpFRInhAll[analysisInd, useSeed, :] = R.trialUpFRInh
        trialUpDurAll[analysisInd, useSeed, :] = R.trialUpDur
#         trialUpFRExcUnitsAll[analysisInd, useSeed, :, :] = R.trialUpFRExcUnits
#         trialUpFRInhUnitsAll[analysisInd, useSeed, :, :] = R.trialUpFRInhUnits
        
        # build a pandas series with all of the above info for the given seed
        scalarQuantVals = (meanDistanceFromSet, meanTerminalDistanceFromSet, meanDistanceFromSetUnits, meanTerminalDistanceFromSetUnits, meanAbsdwEE, meanAbsdwIE, meanAbsdwEI, meanAbsdwII, meanAbsdwSum, meanSumAbsMAvgDiffdW, meanSumAbsMAvgDiffTerminaldW, meanSumAbsMAvgDiffFRMSE, meanSumAbsMAvgDiffTerminalFRMSE, meanAbsTerminaldwEE, meanAbsTerminaldwIE, meanAbsTerminaldwEI, meanAbsTerminaldwII, meanAbsTerminaldwSum, minMSEExc, meanMSEExc, meanTerminalMSEExc, minMSEExcUnits, meanMSEExcUnits, meanTerminalMSEExcUnits, minMSEInh, meanMSEInh, meanTerminalMSEInh, minMSEInhUnits, meanMSEInhUnits, meanTerminalMSEInhUnits, meanTerminalMSEUnits, meanTerminalMSE, meanInitialMSEUnits, meanInitialMSE, firstConvergentInd + 1)
        scalarQuantLbls = ('meanDistanceFromSet', 'meanTerminalDistanceFromSet', 'meanDistanceFromSetUnits', 'meanTerminalDistanceFromSetUnits', 'meanAbsdwEE', 'meanAbsdwIE', 'meanAbsdwEI', 'meanAbsdwII', 'meanAbsdwSum', 'meanSumAbsMAvgDiffdW', 'meanSumAbsMAvgDiffTerminaldW', 'meanSumAbsMAvgDiffFRMSE', 'meanSumAbsMAvgDiffTerminalFRMSE', 'meanAbsTerminaldwEE', 'meanAbsTerminaldwIE', 'meanAbsTerminaldwEI', 'meanAbsTerminaldwII', 'meanAbsTerminaldwSum', 'minMSEExc', 'meanMSEExc', 'meanTerminalMSEExc', 'minMSEExcUnits', 'meanMSEExcUnits', 'meanTerminalMSEExcUnits', 'minMSEInh', 'meanMSEInh', 'meanTerminalMSEInh', 'minMSEInhUnits', 'meanMSEInhUnits', 'meanTerminalMSEInhUnits', 'meanTerminalMSEUnits', 'meanTerminalMSE', 'meanInitialMSEUnits', 'meanInitialMSE', 'trialsToConverge')
        analysisName = (analysisLabels[analysisInd], R.p['rngSeed'])
#         analysisName = (R.p['rngSeed'], R.p['useRule'])

        currentSeries = pd.Series(scalarQuantVals, index=scalarQuantLbls, name=analysisName)
        allSeries.append(currentSeries)

In [42]:
trialwEEAll.shape

(4, 9, 6000)

In [18]:
MSEAll = MSEExcAll + MSEInhAll
MSEUnitsHatAll = (MSEExcUnitsHatAll + MSEInhUnitsHatAll) / 2
frSumAll = frSumExcAll + frSumInhAll
R.rID = 'seedAnalysis'   # this simply defines the label for saved figures

In [19]:
totalEToSingleEAll = trialwEEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleEAll = trialwEIAll * R.p['nInh'] * R.p['propConnect']
totalEToSingleIAll = trialwIEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleIAll = trialwIIAll * R.p['nInh'] * R.p['propConnect']

netCurrentToSingleEAll = totalEToSingleEAll - totalIToSingleEAll
netCurrentToSingleIAll = totalEToSingleIAll - totalIToSingleIAll

In [20]:
overallDF = pd.concat(allSeries, axis=1).transpose()

In [21]:
ruleRenamer = {
    'cross-homeo-pre-scalar': 'CH-global',
    'cross-homeo-pre-scalar-homeo': '2T-global',
    'cross-homeo-pre-outer': 'CH-local',
    'cross-homeo-pre-outer-homeo': '2T-local',
    'cross-homeo-pre-outer-homeo-noLims': '2T-local-noLims',
    'cross-homeo-pre-outer-homeo-noUpperLims': '2T-local-noUpperLims',
    'cross-homeo-pre-scalar-eS': 'CH-G',
    'cross-homeo-pre-outer-eS': 'CH-L',
    'cross-homeo-pre-scalar-homeo-eS': '2T-G',
    'cross-homeo-pre-outer-homeo-eS': '2T-L',
}

ruleValues = overallDF.index.get_level_values(0).to_list()
ruleValuesRenamed = [ruleRenamer[rV] for rV in ruleValues]
ruleTerms = [rVR.split('-')[0] for rVR in ruleValuesRenamed]
ruleScope = [rVR.split('-')[1] for rVR in ruleValuesRenamed]

overallDF['rule'] = ruleValuesRenamed
overallDF['ruleTerms'] = ruleTerms
overallDF['ruleScope'] = ruleScope

In [22]:
# for each measure, do a violinplot showing all 4 learning rules against each other
# to do this we need a small dataframe with two columns: the measure, and the label
# the dataframe ROWS should have all
# so basically this requires making the label (the learning rule) a column

In [23]:
measureLst = overallDF.columns.to_list()[:-1]

In [24]:
selectMeasureLst = ('meanSumAbsMAvgDiffTerminalFRMSE', 'meanSumAbsMAvgDiffTerminaldW', 'trialsToConverge',)

In [25]:
plt.close('all')

In [26]:
fontProp = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')

In [41]:
# plot dedicated to showing the chosen MSE measure for only the 2-factor local, comparing the initial trial vs. the final trial

fontProp.set_size(30)
removeFrame = True
doSave = True

f, ax = plt.subplots(figsize=(6, 6))

tmpDF1 = overallDF.loc['cross-homeo-pre-outer-homeo-eS', :]
tmpDF1.loc[:, 'phase'] = 'Initial'
tmpDF1.loc[:, 'meanTerminalMSEUnits'] = tmpDF1['meanInitialMSEUnits']

tmpDF2 = overallDF.loc['cross-homeo-pre-outer-homeo-eS', :]
tmpDF2.loc[:, 'phase']  = 'Final'

useDF = pd.concat((tmpDF1, tmpDF2))

measureName = 'meanTerminalMSEUnits'  # meanTerminalMSE, meanTerminalMSEUnits, meanAbsTerminaldwSum

yLabeler = {
    'meanTerminalMSE': 'MSE of Pop. Avg. FR',
    'meanTerminalMSEUnits': 'MSE',
    'meanAbsTerminaldwSum': 'MACW',
}

subDF = useDF.loc[:, (measureName, 'phase')]
groupByObj = subDF.groupby('phase', sort=False)
meanVals = groupByObj.mean().values
stdVals = groupByObj.std().values
countVals = groupByObj.count().values
semVals = stdVals / np.sqrt(countVals)
useError = semVals[:, 0]  # or semVals

sns.stripplot(x='phase', y=measureName, data=useDF, color='k', ax=ax, size=4, zorder=1)
sns.pointplot(x='phase', y=measureName, data=useDF, ax=ax, join=False, zorder=2)
ax.set_title(measureName, fontproperties=fontProp)
ax.set_xlabel('Training Phase', fontproperties=fontProp)
ax.set_ylabel(yLabeler[measureName], fontproperties=fontProp)
plt.yscale('log')

for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)

if removeFrame:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
f.subplots_adjust(left=0.2, bottom=0.15)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_' + measureName + '_byPhase.pdf'
    f.savefig(saveFileName, transparent=True)

WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
 [py.warnings]


In [15]:
# plot dedicated to showing the chosen MSE measure

fontProp.set_size(24)
removeFrame = True
doSave = False

f, ax = plt.subplots(figsize=(6, 6))

measureName = 'meanAbsTerminaldwSum'  # meanTerminalMSE, meanTerminalMSEUnits, meanAbsTerminaldwSum

yLabeler = {
    'meanTerminalMSE': 'MSE of Pop. Avg. FR',
    'meanTerminalMSEUnits': 'MSE',
    'meanAbsTerminaldwSum': 'MACW',
}

subDF = overallDF.loc[:, (measureName, 'rule')]
groupByObj = subDF.groupby('rule', sort=False)
meanVals = groupByObj.mean().values
stdVals = groupByObj.std().values
countVals = groupByObj.count().values
semVals = stdVals / np.sqrt(countVals)
useError = semVals[:, 0]  # or semVals

# sns.barplot(x='rule', y=measureName, data=overallDF, ax=ax)
sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=ax, size=4, zorder=1)
sns.pointplot(x='rule', y=measureName, data=overallDF, ax=ax, join=False, zorder=2)
# ax.scatter(np.arange(meanVals.size), meanVals, 50, zorder=3)
# ax.errorbar(np.arange(meanVals.size), meanVals, yerr=useError, ls='none', zorder=2)
ax.set_title(measureName, fontproperties=fontProp)
ax.set_xlabel('Rule Family', fontproperties=fontProp)
ax.set_ylabel(yLabeler[measureName], fontproperties=fontProp)
plt.yscale('log')

for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)

if removeFrame:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
f.subplots_adjust(left=0.2, bottom=0.15)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_' + measureName + '.pdf'
    f.savefig(saveFileName, transparent=True)

In [62]:
semVals[:, 0]

array([0.18779136, 0.18740297, 0.20791138, 2.32073665])

In [49]:
measureLst

['meanDistanceFromSet',
 'meanTerminalDistanceFromSet',
 'meanDistanceFromSetUnits',
 'meanTerminalDistanceFromSetUnits',
 'meanAbsdwEE',
 'meanAbsdwIE',
 'meanAbsdwEI',
 'meanAbsdwII',
 'meanAbsdwSum',
 'meanSumAbsMAvgDiffdW',
 'meanSumAbsMAvgDiffTerminaldW',
 'meanSumAbsMAvgDiffFRMSE',
 'meanSumAbsMAvgDiffTerminalFRMSE',
 'meanAbsTerminaldwEE',
 'meanAbsTerminaldwIE',
 'meanAbsTerminaldwEI',
 'meanAbsTerminaldwII',
 'meanAbsTerminaldwSum',
 'minMSEExc',
 'meanMSEExc',
 'meanTerminalMSEExc',
 'minMSEExcUnits',
 'meanMSEExcUnits',
 'meanTerminalMSEExcUnits',
 'minMSEInh',
 'meanMSEInh',
 'meanTerminalMSEInh',
 'minMSEInhUnits',
 'meanMSEInhUnits',
 'meanTerminalMSEInhUnits',
 'meanTerminalMSEUnits',
 'trialsToConverge']

In [51]:
# plot dedicated to showing the mean MSE, for the average FR and the unit FR

f, ax = plt.subplots(2, 2, sharex=True, sharey=False, figsize=(10, 10))
axr = ax.ravel()

# weightMeasures = ('meanMSEExc', 'meanMSEInh', 'meanMSEExcUnits','meanMSEInhUnits',)
weightMeasures = ('meanTerminalMSEExc', 'meanTerminalMSEInh', 'meanTerminalMSEExcUnits', 'meanTerminalMSEInhUnits',)

for measureInd, measureName in enumerate(weightMeasures):
    sns.barplot(x='rule', y=measureName, data=overallDF, ax=axr[measureInd])
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
    sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
#     sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
    axr[measureInd].set(title=measureName)
    if measureInd <= 1:
        axr[measureInd].set(xlabel='')
    if (measureInd % 2) == 1:
        axr[measureInd].set(ylabel='')
    else:
        axr[measureInd].set(ylabel='MSE')
    axr[measureInd].set_yscale('log')

In [25]:
from statsmodels import stats

In [28]:
dir(stats)

['PytestTester',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_adnorm',
 '_knockoff',
 '_lilliefors',
 '_lilliefors_critical_values',
 'anova',
 'api',
 'base',
 'contingency_tables',
 'contrast',
 'correlation_tools',
 'descriptivestats',
 'diagnostic',
 'gof',
 'mediation',
 'moment_helpers',
 'multicomp',
 'multitest',
 'oaxaca',
 'power',
 'proportion',
 'sandwich_covariance',
 'stattools',
 'tabledist',
 'test',
 'weightstats']

In [20]:
import statsmodels

In [21]:
statsmodels.__version__

'0.12.2'

In [12]:
# plot dedicated to showing the euclidean distance measures

f, ax = plt.subplots(2, 2, sharex=True, sharey=False, figsize=(10, 10))
axr = ax.ravel()

weightMeasures = ('meanDistanceFromSet', 'meanTerminalDistanceFromSet', 'meanDistanceFromSetUnits', 'meanTerminalDistanceFromSetUnits',)

for measureInd, measureName in enumerate(weightMeasures):
    sns.barplot(x='rule', y=measureName, data=overallDF, ax=axr[measureInd])
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
    sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
#     sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
    axr[measureInd].set(title=measureName)
    if measureInd <= 1:
        axr[measureInd].set(xlabel='')
    if (measureInd % 2) == 1:
        axr[measureInd].set(ylabel='')
    else:
        axr[measureInd].set(ylabel='Euclidean distance')

In [61]:
# plot dedicated to showing the chosen MACW measure

f, ax = plt.subplots(figsize=(5, 6))

measureName = 'meanAbsTerminaldwSum'

sns.barplot(x='rule', y=measureName, data=overallDF, ax=ax)
sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=ax)
ax.set(title=measureName)
ax.set(ylabel='MACW')
plt.yscale('log')

In [13]:
# plot dedicated to showing the mean(abs(dW)) for all 4 weight classes
f, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 10))
axr = ax.ravel()

# weightMeasures = ('meanAbsdwEE', 'meanAbsdwIE', 'meanAbsdwEI', 'meanAbsdwII',)
weightMeasures = ('meanAbsTerminaldwEE', 'meanAbsTerminaldwIE', 'meanAbsTerminaldwEI', 'meanAbsTerminaldwII',)

for measureInd, measureName in enumerate(weightMeasures):
    sns.barplot(x='rule', y=measureName, data=overallDF, ax=axr[measureInd])
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
    sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
#     sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
    axr[measureInd].set(title=measureName)
    if measureInd <= 1:
        axr[measureInd].set(xlabel='')
    if (measureInd % 2) == 1:
        axr[measureInd].set(ylabel='')
    else:
        axr[measureInd].set(ylabel='mean(abs(dW))')

axr[measureInd].set(ylim=(0, 0.3))

[(0.0, 0.3)]

In [15]:
# the rest

for measureInd, measureName in enumerate(selectMeasureLst):
    f, ax = plt.subplots()
    sns.barplot(x='rule', y=measureName, data=overallDF)
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
#     sns.stripplot(x='rule', y=measureName, data=overallDF, color='k')
    sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k')

# NON BAR GRAPHS

In [66]:
# plot that shows weight convergence of all 4 learning rules on one graph for one seed

movAvgWidth = 31  # 31
useSeed = 2

f, ax = plt.subplots(2, 2, sharex=True, sharey=True)
axr = ax.ravel()

for analysisInd, analysisLabel in enumerate(analysisLabels):

    axr[analysisInd].plot(trialwEEAll[analysisInd, useSeed, :], label='wEE', color='cyan')
    axr[analysisInd].plot(trialwIEAll[analysisInd, useSeed, :], label='wIE', color='purple')
    axr[analysisInd].plot(trialwEIAll[analysisInd, useSeed, :], label='wEI', color='cyan', ls='--')
    axr[analysisInd].plot(trialwIIAll[analysisInd, useSeed, :], label='wII', color='purple', ls='--')
    axr[analysisInd].legend()
    axr[analysisInd].set_xlabel('Trial #')
    axr[analysisInd].set_ylabel('Weight (pA)')
    axr[analysisInd].set(ylabel='FR (Hz)', title=ruleRenamer[analysisLabel])

In [67]:
# plot that shows E/I FR convergence of all 4 learning rules on one graph for one seed

movAvgWidth = 31  # 31
useSeed = 2

f, ax = plt.subplots(2, 2, sharex=True, sharey=True)
axr = ax.ravel()

for analysisInd, analysisLabel in enumerate(analysisLabels):

    if movAvgWidth:
        useFRExc = moving_average(trialUpFRExcAll[analysisInd, useSeed, :], movAvgWidth)
        useFRInh = moving_average(trialUpFRInhAll[analysisInd, useSeed, :], movAvgWidth)
    else:
        useFRExc = trialUpFRExcAll[analysisInd, useSeed, :]
        useFRInh = trialUpFRInhAll[analysisInd, useSeed, :]

    axr[analysisInd].hlines(R.p['setUpFRExc'], 0, nTrials, ls='--', color='g')
    axr[analysisInd].hlines(R.p['setUpFRInh'], 0, nTrials, ls='--', color='r')
    axr[analysisInd].plot(useFRExc, label='E', color='g', alpha=0.7)
    axr[analysisInd].plot(useFRInh, label='I', color='r', alpha=0.7)
    axr[analysisInd].set(ylabel='FR (Hz)', title=ruleRenamer[analysisLabel])

axr[analysisInd].set(ylabel='FR (Hz)', ylim=(0, 20))

[Text(0, 0.5, 'FR (Hz)'), (0.0, 20.0)]

In [68]:
# plot that shows E/I FR convergence of all 4 learning rules on one graph for all seeds

movAvgWidth = 31  # 31

f, ax = plt.subplots(2, 2, sharex=True, sharey=True)
axr = ax.ravel()

for useSeed in range(9):
    for analysisInd, analysisLabel in enumerate(analysisLabels):

        if movAvgWidth:
            useFRExc = moving_average(trialUpFRExcAll[analysisInd, useSeed, :], movAvgWidth)
            useFRInh = moving_average(trialUpFRInhAll[analysisInd, useSeed, :], movAvgWidth)
        else:
            useFRExc = trialUpFRExcAll[analysisInd, useSeed, :]
            useFRInh = trialUpFRInhAll[analysisInd, useSeed, :]

        axr[analysisInd].hlines(R.p['setUpFRExc'], 0, nTrials, ls='--', color='g')
        axr[analysisInd].hlines(R.p['setUpFRInh'], 0, nTrials, ls='--', color='r')
        axr[analysisInd].plot(useFRExc, label='E', color='g', alpha=0.7)
        axr[analysisInd].plot(useFRInh, label='I', color='r', alpha=0.7)
        axr[analysisInd].set(ylabel='FR (Hz)', title=ruleRenamer[analysisLabel])

    axr[analysisInd].set(ylabel='FR (Hz)', ylim=(0, 20))

In [70]:
# plot that shows E/I FR convergence AVERAGED ACROSS ALL SEEDS of all 4 learning rules

movAvgWidth = 31  # 31

f, ax = plt.subplots(2, 2, sharex=True, sharey=True)
axr = ax.ravel()

for analysisInd, analysisLabel in enumerate(analysisLabels):

    if movAvgWidth:
        useFRExc = moving_average(trialUpFRExcAll.mean(1)[analysisInd, :], movAvgWidth)
        useFRInh = moving_average(trialUpFRInhAll.mean(1)[analysisInd, :], movAvgWidth)
    else:
        useFRExc = trialUpFRExcAll.mean(1)[analysisInd, :]
        useFRInh = trialUpFRInhAll.mean(1)[analysisInd, :]

    axr[analysisInd].hlines(R.p['setUpFRExc'], 0, nTrials, ls='--', color='g')
    axr[analysisInd].hlines(R.p['setUpFRInh'], 0, nTrials, ls='--', color='r')
    axr[analysisInd].plot(useFRExc, label='E', color='g', alpha=0.7)
    axr[analysisInd].plot(useFRInh, label='I', color='r', alpha=0.7)
    axr[analysisInd].set(ylabel='FR (Hz)', title=ruleRenamer[analysisLabel])

axr[analysisInd].set(ylabel='FR (Hz)', ylim=(0, 20))

[Text(0, 0.5, 'FR (Hz)'), (0.0, 20.0)]

In [119]:
# plot that shows convergence of all 9 seeds on one graph for all learning rules

doSave = True

useData = frSumAll  # frSumAll, wSumAll, MSEAll

f, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(8, 6))
axr = ax.ravel()

useSeedsLocal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
# useSeedsLocal = (0, 1, 3, 4, 5, 7, 8)

for analysisInd, analysisLabel in enumerate(analysisLabels):
    for useSeed in useSeedsLocal:
        axr[analysisInd].plot(useData[analysisInd, useSeed, :], label=str(useSeed), alpha=.5)
    plt.yscale('log')
    axr[analysisInd].set(title=ruleRenamer[analysisLabel], ylim=(1e1, useData.max()))
    if analysisInd == 2:
        axr[analysisInd].legend()
        
if doSave:
    saveFileName = 'figs/' + R.rID + '_MSEOverTrialsAll.pdf'
    f.savefig(saveFileName, transparent=True)

In [83]:
# plot that shows the progression of Up duration of all 9 seeds on one graph for all learning rules

movAvgWidth = 31  # 31
useData = trialUpDurAll  # frSumAll, wSumAll, MSEAll

f, ax = plt.subplots(2, 2, sharex=True, sharey=True)
axr = ax.ravel()

useSeedsLocal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
# useSeedsLocal = (0, 1, 3, 4, 5, 7, 8)

for analysisInd, analysisLabel in enumerate(analysisLabels):
    for useSeed in useSeedsLocal:
        if movAvgWidth:
            useDataLocal = moving_average(useData[analysisInd, useSeed, :], movAvgWidth)
        else:
            useDataLocal = useData[analysisInd, useSeed, :]
        axr[analysisInd].plot(useDataLocal, label=str(useSeed), alpha=.5)
    axr[analysisInd].set(title=ruleRenamer[analysisLabel], ylim=(0, useData.max()))
    if analysisInd == 2:
        axr[analysisInd].legend()

In [120]:
# plot that shows convergence of all 4 learning rules on one graph for one seed
# note that frSumAll is an MSE measure

doSave = True

useData = frSumAll  # frSumAll, wSumAll, MSEAll, distanceFromSetAll, distanceFromSetUnitsAll
useMovAvg = True
useLogYScale = True
movAvgWidth = 31
useYLabel = ''

f, ax = plt.subplots()

useSeed = 8
for analysisInd, analysisLabel in enumerate(analysisLabels):
    useFirstConvergentInd = int(firstConvergentIndAll[analysisInd, useSeed])
    yData = useData[analysisInd, useSeed, :]
    if useMovAvg:
        yData = moving_average(yData, movAvgWidth)
    ax.plot(yData, label=ruleRenamer[analysisLabel], alpha=0.7)
if useLogYScale:
    plt.yscale('log')
ax.legend()
ax.set(xlabel='Trial #', ylabel=useYLabel)

if doSave:
    saveFileName = 'figs/' + R.rID + '_MSEOverTrialsHat.pdf'
    f.savefig(saveFileName, transparent=True)

In [89]:
# plot that shows convergence of all 4 learning rules on one graph for each seed

useData = frSumAll  # frSumAll, wSumAll, MSEAll, distanceFromSetAll, distanceFromSetUnitsAll
useMovAvg = True
useLogYScale = True
movAvgWidth = 31
useYLabel = ''
useYLims = (1e1, useData.max())
# useYLims = (0, 1)

f, ax = plt.subplots(3, 3, sharex=True, sharey=True)
axr = ax.ravel()

for useSeed in useSeeds:
    for analysisInd, analysisLabel in enumerate(analysisLabels):
        useFirstConvergentInd = int(firstConvergentIndAll[analysisInd, useSeed])
        yData = useData[analysisInd, useSeed, :]
        if useMovAvg:
            yData = moving_average(yData, movAvgWidth)
        axr[useSeed].plot(yData, label=ruleRenamer[analysisLabel], alpha=0.5)
    if useLogYScale:
        plt.yscale('log')
    axr[useSeed].set(title=str(useSeed), ylim=useYLims)
    if useSeed == 2:
        axr[useSeed].legend()
    if useSeed % 3 == 0:
        axr[useSeed].set(ylabel=useYLabel)
    if useSeed >= 6:
        axr[useSeed].set(xlabel='Trial #')

In [95]:
# original way, net current

totalEToSingleEAll = trialwEEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleEAll = trialwEIAll * R.p['nInh'] * R.p['propConnect']
totalEToSingleIAll = trialwIEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleIAll = trialwIIAll * R.p['nInh'] * R.p['propConnect']

# make the numbers smaller for crybaby Dean

netCurrentToSingleEAll = totalEToSingleEAll - totalIToSingleEAll
netCurrentToSingleIAll = totalEToSingleIAll - totalIToSingleIAll

# netCurrentToSingleEAll = (totalEToSingleEAll - totalIToSingleEAll) / R.p['nUnits'] * R.p['propConnect']
# netCurrentToSingleIAll = (totalEToSingleIAll - totalIToSingleIAll) / R.p['nUnits'] * R.p['propConnect']

In [22]:
# different way, weights

netCurrentToSingleEAll = trialwEEAll - trialwEIAll
netCurrentToSingleIAll = trialwIEAll - trialwIIAll

In [96]:
# plot that shows INITIAL CONDITIONS of all 9 seeds on one graph for only one learning rule, but in a 2D system defined by
# the net current to each E unit (x) vs. the net current to each I unit (y)

# useSeedsLocal = (0, 1, 3, 4, 5, 7, 8,)
useSeedsLocal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
useColors = ('magenta', 'red', 'grey', 'orange', 'green', 'blue', 'purple', 'cyan', 'black')
useAnalysisInd = 1

f, ax = plt.subplots(figsize=(8, 5))
for useSeed in useSeedsLocal:
    ax.scatter(netCurrentToSingleEAll[useAnalysisInd, useSeed, 0], netCurrentToSingleIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')

ax.set(xlabel='Net weight onto each E unit (on average)', ylabel='Net weight onto each I unit (on average)')

[Text(0.5, 0, 'Net weight onto each E unit (on average)'),
 Text(0, 0.5, 'Net weight onto each I unit (on average)')]

In [97]:
from stats import regress_linear

In [98]:
trialwEEAll.shape

(4, 9, 6000)

In [104]:
usewEEAll = trialwEEAll * R.p['nExc'] * R.p['propConnect']
usewEIAll = trialwEIAll * R.p['nInh'] * R.p['propConnect']
usewIEAll = trialwIEAll * R.p['nExc'] * R.p['propConnect']
usewIIAll = trialwIIAll * R.p['nInh'] * R.p['propConnect']

In [107]:
usewEEAll = trialwEEAll
usewEIAll = trialwEIAll
usewIEAll = trialwIEAll
usewIIAll = trialwIIAll

In [132]:
analysisLabels

('cross-homeo-pre-scalar-eS',
 'cross-homeo-pre-scalar-homeo-eS',
 'cross-homeo-pre-outer-eS',
 'cross-homeo-pre-outer-homeo-eS')

In [147]:
# plot that shows convergence of all 9 seeds on one graph for only one learning rule, but in a 2D system defined by
# the net current to each E unit (x) vs. the net current to each I unit (y)

fontProp.set_size(16)
removeFrame = True
doSave = True

# useSeedsLocal = (0, 1, 3, 4, 5, 7, 8,)
useSeedsLocal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
useColors = ('magenta', 'red', 'grey', 'orange', 'green', 'blue', 'purple', 'cyan', 'black')
useAnalysisInd = 1

xVals = usewEEAll[useAnalysisInd, useSeedsLocal, -1]
yVals = usewEIAll[useAnalysisInd, useSeedsLocal, -1]

pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)

f, ax = plt.subplots(1, 2, figsize=(11, 5))

ax[0].plot(pred_x, pred_y, color='k')
for useSeed in useSeedsLocal:
    ax[0].plot(usewEEAll[useAnalysisInd, useSeed, :], usewEIAll[useAnalysisInd, useSeed, :],
            label=str(useSeed), alpha=.5, color=useColors[useSeed])
    ax[0].scatter(usewEEAll[useAnalysisInd, useSeed, 0], usewEIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')
    ax[0].scatter(usewEEAll[useAnalysisInd, useSeed, -1], usewEIAll[useAnalysisInd, useSeed, -1], 30,
               color=useColors[useSeed], marker='v')

# ax[0].axes.xaxis.set_ticks([])
# ax[0].axes.yaxis.set_ticks([])

ax[0].set_xlabel('mean J EE (pA)', fontproperties=fontProp)
ax[0].set_ylabel('mean J EI (pA)', fontproperties=fontProp)
# ax[0].legend()

xVals = usewIEAll[useAnalysisInd, useSeedsLocal, -1]
yVals = usewIIAll[useAnalysisInd, useSeedsLocal, -1]

pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)

ax[1].plot(pred_x, pred_y, color='k')
for useSeed in useSeedsLocal:
    ax[1].plot(usewIEAll[useAnalysisInd, useSeed, :], usewIIAll[useAnalysisInd, useSeed, :],
            label=str(useSeed), alpha=.5, color=useColors[useSeed])
    ax[1].scatter(usewIEAll[useAnalysisInd, useSeed, 0], usewIIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')
    ax[1].scatter(usewIEAll[useAnalysisInd, useSeed, -1], usewIIAll[useAnalysisInd, useSeed, -1], 30,
               color=useColors[useSeed], marker='v')

# ax[1].axes.xaxis.set_ticks([])
# ax[1].axes.yaxis.set_ticks([])

ax[1].set_xlabel('mean J IE (pA)', fontproperties=fontProp)
ax[1].set_ylabel('mean J II (pA)', fontproperties=fontProp)
ax[1].legend(frameon=False, prop=fontProp, labelspacing=0.25)

for axr in ax:
    for xtl in axr.get_xticklabels():
        xtl.set_fontproperties(fontProp)
    for ytl in axr.get_yticklabels():
        ytl.set_fontproperties(fontProp)

    if removeFrame:
        axr.spines['top'].set_visible(False)
        axr.spines['right'].set_visible(False)

if doSave:
    saveFileName = 'figs/' + R.rID + '_2DConverge.pdf'
    f.savefig(saveFileName, transparent=True)

In [106]:
# plot that shows convergence of all 9 seeds on one graph for only one learning rule, but in a 2D system defined by
# the wEE and wEI

useSeedsLocal = (0, 1, 3, 4, 5, 7, 8,)
# useSeedsLocal = (0, 1, 2 3, 4, 5, 6, 7, 8)
useColors = ('magenta', 'red', 'grey', 'orange', 'green', 'blue', 'purple', 'cyan', 'black')
useAnalysisInd = 1

xVals = netCurrentToSingleEAll[useAnalysisInd, useSeedsLocal, -1]
yVals = netCurrentToSingleIAll[useAnalysisInd, useSeedsLocal, -1]

pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)

f, ax = plt.subplots(figsize=(9, 7))
ax.plot(pred_x, pred_y, color='k')
for useSeed in useSeedsLocal:
    ax.plot(netCurrentToSingleEAll[useAnalysisInd, useSeed, :], netCurrentToSingleIAll[useAnalysisInd, useSeed, :],
            label=str(useSeed), alpha=.5, color=useColors[useSeed])
    ax.scatter(netCurrentToSingleEAll[useAnalysisInd, useSeed, 0], netCurrentToSingleIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')
    ax.scatter(netCurrentToSingleEAll[useAnalysisInd, useSeed, -1], netCurrentToSingleIAll[useAnalysisInd, useSeed, -1], 30,
               color=useColors[useSeed], marker='v')
ax.hlines(0, *ax.get_xlim(), color='k', linestyle='--', alpha=0.2)
ax.vlines(0, *ax.get_ylim(), color='k', linestyle='--', alpha=0.2)

ax.axes.xaxis.set_ticks([])
ax.axes.yaxis.set_ticks([])

ax.set(xlabel='Mean synaptic current onto E units', ylabel='Mean synaptic current onto I units')
ax.legend()

In [ ]:
# additional measure: # of trials to convergence (double check!!!)

In [ ]:
# additional measures: pre-convergent values

In [150]:
overallDF

meanDistanceFromSet  \
cross-homeo-pre-scalar-eS       0             7.476634   
                                1             1.661566   
                                2             0.891123   
                                3             3.728204   
                                4             4.709907   
                                5             0.620190   
                                6             4.153634   
                                7             3.836416   
                                8             4.970965   
cross-homeo-pre-scalar-homeo-eS 0             8.619218   
                                1             1.986164   
                                2             2.078207   
                                3             4.639288   
                                4             3.895222   
                                5             0.961074   
                                6             6.526534   
                                7             4.785437   
                                8             6.916776   
cross-homeo-pre-outer-eS        0            14.199907   
                                1             9.984837   
                                2             6.780028   
                                3            12.269550   
                                4            11.659086   
                                5            11.127018   
                                6             9.747143   
                                7            12.224898   
                                8            10.998323   
cross-homeo-pre-outer-homeo-eS  0             9.303744   
                                1             1.962566   
                                2             2.078391   
                                3             4.243772   
                                4             3.964329   
                                5             0.961124   
                                6             6.465687   
                                7             4.629864   
                                8             6.246271   

                                   meanTerminalDistanceFromSet  \
cross-homeo-pre-scalar-eS       0                     2.345602   
                                1                     0.346380   
                                2                     0.255404   
                                3                     0.367657   
                                4                     1.812491   
                                5                     0.248097   
                                6                     0.317272   
                                7                     2.238139   
                                8                     1.551196   
cross-homeo-pre-scalar-homeo-eS 0                     2.253462   
                                1                     0.298899   
                                2                     0.204709   
                                3                     0.600959   
                                4                     1.724198   
                                5                     0.203951   
                                6                     2.149625   
                                7                     2.075996   
                                8                     2.059955   
cross-homeo-pre-outer-eS        0                    14.868331   
                                1                    10.821089   
                                2                     9.326583   
                                3                    12.688492   
                                4                    11.931803   
                                5                    13.219140   
                                6                    12.266017   
                                7                    14.816731   
                                8                    12.000686   
cross-homeo-pre-outer-homeo-eS  0       

In [160]:
overallDF.to_csv(os.getcwd() + '\\seedResultsLims.csv')

In [72]:
overallDF.groupby(level=0).mean()

,meanDistanceFromSet,meanTerminalDistanceFromSet,meanDistanceFromSetUnits,meanTerminalDistanceFromSetUnits,meanAbsdwEE,meanAbsdwIE,meanAbsdwEI,meanAbsdwII,meanAbsdwSum,meanSumAbsMAvgDiffdW,...,meanMSEExcUnits,meanTerminalMSEExcUnits,minMSEInh,meanMSEInh,meanTerminalMSEInh,minMSEInhUnits,meanMSEInhUnits,meanTerminalMSEInhUnits,meanTerminalMSEUnits,trialsToConverge
cross-homeo-pre-outer-homeo,2.727548,0.548754,206.101493,108.928273,0.062682,0.030107,0.164325,0.076298,0.333412,0.096848,...,209.903952,3.835842,9.015553e-31,92.033062,0.771550,10.462571,200.883737,15.424205,9.630024,4419.857143
cross-homeo-pre-outer-homeo-noLims,10.507891,5.831694,303.920541,239.092555,0.030768,0.032120,0.137434,0.056689,0.257011,0.090143,...,25.148133,14.336241,2.727407e+01,130.345054,60.550947,80.714927,162.764423,123.118753,68.727497,4.857143


# TRAINING ANALYSIS

In [31]:
plt.close('all')

In [17]:
basePath = 'C:/Users/mikejseay/Documents/BrianResults/'

# analysisDict = crossHomeoPreScalar.copy()  # seed 6/7 is good examples
# analysisDict = crossHomeoPreScalarHomeo.copy()
# analysisDict = crossHomeoPreOuter.copy()
# analysisDict = crossHomeoPreOuterHomeo.copy()
# analysisDict = crossHomeoPreOuterHomeoNoLims.copy()
# analysisDict = crossHomeoPreOuterHomeoNoUpperLims.copy()
# analysisDict = crossHomeoPreScalarHomeoRemean.copy()
# analysisDict = crossHomeoPreScalarExplodeSolve.copy()
analysisDict = crossHomeoPreOuterHomeoExplodeSolve.copy()

useSeed = 8

targetPath = basePath + analysisDict['subfolder']

if len(analysisDict[useSeed]) == 2:
    targetFile1, targetFile2 = analysisDict[useSeed]
    R1 = Results()
    R1.init_from_file(targetFile1, targetPath)
    R2 = Results()
    R2.init_from_file(targetFile2, targetPath)
    R = combine_two_sessions(R1, R2)
else:
    targetFile = analysisDict[useSeed]
    R = Results()
    R.init_from_file(targetFile, targetPath)

print(R.rID)

# single files
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_goodCrossHomeoExamp__2022-01-26-09-16-00_results'
# R = Results()
# R.init_from_file(targetFile, basePath)

print(R.p['setUpFRExc'], R.p['tauUpFRTrials'], R.p['alpha1'], R.p['spikeInputAmplitude'])
# plt.close('all')
# pdfObject = PdfPages(targetPath + '/reports/' + R.rID + '_report.pdf')
pdfObject = PdfPages(basePath + '/reports/' + R.rID + '_report.pdf')

removeOutliers = False

generate_description(R)

FR_weights(R)
# FR_weights(R, movAvgWidth=31)
# plt.savefig(pdfObject, format='pdf')

# firstConvergentInd = int(R.trialwEE.size / 2)
# firstConvergentInd = 4700
firstConvergentInd = determine_first_convergent_trial(R)
# firstConvergentInd = determine_first_convergent_trial(R) + 1500
print('We think the first convergent index is', firstConvergentInd)

classicJercog_2000_0p25_cross-homeo-pre-outer-homeo_seed8_explodeDealTestLowAlpha_2022-02-02-23-09-50
5. Hz 2 2.5 fC 0.98
We think the first convergent index is 2552


In [ ]:
determine_drift_rates(R, firstConvergentInd)
print(R.driftRateEE)
print(R.driftRateEI)
print(R.driftRateIE)
print(R.driftRateII)

# convergence process
FR_scatter(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
FR_hist1d_compare(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
FR_weights_std(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
weights_scatter_matrix(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')

# post convergence
FR_scatter(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
plt.savefig(pdfObject, format='pdf')
FR_hist2d(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
plt.savefig(pdfObject, format='pdf')
FR_hist1d_compare(R, startTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
FR_weights_std(R, startTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
weights_scatter_matrix(R, startTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')

# epilogue
weights_hist1d_compare(R)
plt.savefig(pdfObject, format='pdf')
weights_matrix_compare(R)
plt.savefig(pdfObject, format='pdf')

pdfObject.close()

In [221]:
plt.close('all')

In [10]:
# goals:
# quantify the MSE at each trial
# quantify the mean(abs(dWEE)) at each step..
# create a common plot across multiple seeds

In [144]:
# preparatory calculations

MSEExc = (R.trialUpFRExc - R.p['setUpFRExc'] / Hz) ** 2
MSEInh = (R.trialUpFRInh - R.p['setUpFRInh'] / Hz) ** 2
MSEExcUnits = (R.trialUpFRExcUnits - R.p['setUpFRExc'] / Hz) ** 2
MSEExcUnitsHat = MSEExcUnits.mean(1)
MSEInhUnits = (R.trialUpFRInhUnits - R.p['setUpFRInh'] / Hz) ** 2
MSEInhUnitsHat = MSEInhUnits.mean(1)
trialdwEE = np.ediff1d(R.trialwEE)
trialdwIE = np.ediff1d(R.trialwIE)
trialdwEI = np.ediff1d(R.trialwEI)
trialdwII = np.ediff1d(R.trialwII)
allAbsdW = np.fabs(trialdwEE) + np.fabs(trialdwIE) + np.fabs(trialdwEI) + np.fabs(trialdwII)
wSum = calculate_convergence_index(R)
frSum = moving_average(MSEExcUnits.mean(1), movAvgWidth) + moving_average(MSEInhUnits.mean(1), movAvgWidth)
movAvgWidth=31
frSumExc = moving_average(MSEExcUnits.mean(1), movAvgWidth)
frSumInh = moving_average(MSEInhUnits.mean(1), movAvgWidth)
frSum = frSumExc + frSumInh

In [145]:
# scalar quantifiers

meanAbsdwEE = np.fabs(trialdwEE).mean()
meanAbsdwIE = np.fabs(trialdwIE).mean()
meanAbsdwEI = np.fabs(trialdwEI).mean()
meanAbsdwII = np.fabs(trialdwII).mean()
meanSumAbsMAvgDiffdW = wSum.mean()
meanSumAbsMAvgDiffConvergentdW = wSum[firstConvergentInd:].mean()
meanSumAbsMAvgDiffFRMSE = frSum.mean()
meanSumAbsMAvgDiffConvergentFRMSE = frSum[firstConvergentInd:].mean()
meanAbsPostConvergencedwEE = np.fabs(trialdwEE[firstConvergentInd:]).mean()
meanAbsPostConvergencedwIE = np.fabs(trialdwIE[firstConvergentInd:]).mean()
meanAbsPostConvergencedwEI = np.fabs(trialdwEI[firstConvergentInd:]).mean()
meanAbsPostConvergencedwII = np.fabs(trialdwII[firstConvergentInd:]).mean()
minMSEExc = MSEExc.min()
meanMSEExc = MSEExc.mean()
meanConvergentMSEExc = MSEExc[firstConvergentInd:].mean()
minMSEExcUnits = MSEExcUnitsHat.min()
meanMSEExcUnits = MSEExcUnitsHat.mean()
meanConvergentMSEExcUnits = MSEExcUnitsHat[firstConvergentInd:].mean()
minMSEInh = MSEInh.min()
meanMSEInh = MSEInh.mean()
meanConvergentMSEInh = MSEInh[firstConvergentInd:].mean()
minMSEInhUnits = MSEInhUnitsHat.min()
meanMSEInhUnits = MSEInhUnitsHat.mean()
meanConvergentMSEInhUnits = MSEInhUnitsHat[firstConvergentInd:].mean()

In [146]:
plt.close('all')

In [140]:
# graphs

f, ax = plt.subplots()
ax.plot(MSEExcUnitsHat, color='green', linewidth=1)
ax.plot(MSEInhUnitsHat, color='red', linewidth=1)
plt.yscale('log')
ax.legend(('Exc', 'Inh'))
ax.set(xlabel='Trial #', ylabel='MSE for avg. FR (Hz ^ 2)')

[Text(0, 0.5, 'MSE for avg. FR (Hz ^ 2)'), Text(0.5, 0, 'Trial #')]

In [131]:
minVal = 1e-6

zData = np.arange(MSEExc.size) + 2

f, ax = plt.subplots()
ax.scatter(MSEExc, MSEInh, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
ax.set(xlim=(minVal, MSEExc.max()), ylim=(minVal, MSEInh.max()),
       xlabel='MSE for avg. E FR (Hz ^ 2)', ylabel='MSE for avg. I FR (Hz ^ 2)')

[(1e-06, 101964.4336322184),
 Text(0, 0.5, 'MSE for avg. I FR (Hz ^ 2)'),
 (1e-06, 14018.514809196724),
 Text(0.5, 0, 'MSE for avg. E FR (Hz ^ 2)')]

In [132]:
minVal = 1e0

zData = np.arange(MSEExcUnitsHat.size) + 2

f, ax = plt.subplots()
ax.scatter(MSEExcUnitsHat, MSEInhUnitsHat, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
ax.set(xlim=(minVal, MSEExcUnitsHat.max()), ylim=(minVal, MSEInhUnitsHat.max()),
       xlabel='MSE for avg. E FR (Hz ^ 2)', ylabel='MSE for avg. I FR (Hz ^ 2)')

[(1.0, 148110.62455567857),
 Text(0, 0.5, 'MSE for avg. I FR (Hz ^ 2)'),
 (1.0, 20629.2448720937),
 Text(0.5, 0, 'MSE for avg. E FR (Hz ^ 2)')]

In [141]:
f, ax = plt.subplots()
ax.plot(frSumExc, color='green')
ax.plot(frSumInh, color='red')
# ax.plot(frSum, color='blue')
plt.yscale('log')
ax.legend(('Exc', 'Inh', 'Exc+Inh'))
ax.set(xlabel='Trial #', ylabel='MovAvg of MSE for avg. FR (Hz ^ 2)')

[Text(0, 0.5, 'MovAvg of MSE for avg. FR (Hz ^ 2)'), Text(0.5, 0, 'Trial #')]

In [152]:
movAvgWidth = 31

f, ax = plt.subplots()
# ax.plot(moving_average(wSum, movAvgWidth))
ax.plot(allAbsdW)
ax.set(xlabel='Trial #', ylabel='Abs. MovAvg. of dW (pA)')

[Text(0, 0.5, 'Abs. MovAvg. of dW (pA)'), Text(0.5, 0, 'Trial #')]

In [151]:
minVal = 1e0

zData = np.arange(frSumExc.size) + 2

f, ax = plt.subplots()
ax.scatter(frSumExc, frSumInh, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
ax.set(xlim=(minVal, MSEExcUnitsHat.max()), ylim=(minVal, MSEInhUnitsHat.max()),
       xlabel='MSE for movAvg. E FR (Hz ^ 2)', ylabel='MSE for movAvg. I FR (Hz ^ 2)')

[(1.0, 148110.62455567857),
 Text(0, 0.5, 'MSE for movAvg. I FR (Hz ^ 2)'),
 (1.0, 20629.2448720937),
 Text(0.5, 0, 'MSE for movAvg. E FR (Hz ^ 2)')]

In [93]:
# there is not enough information to calculate mean(abs(dwEE)) but it could be re-calculated like this...

nTrials = R.trialUpFRExcUnits.shape[0]

trialMAdwEE = np.empty((nTrials,))
trialMAdwIE = np.empty((nTrials,))
trialMAdwEI = np.empty((nTrials,))
trialMAdwII = np.empty((nTrials,))

# get some adjacency matrix and nPre
aEE = adjacency_matrix_from_flat_inds(R.p['nExc'], R.p['nExc'], R.preEE, R.posEE)
aEI = adjacency_matrix_from_flat_inds(R.p['nInh'], R.p['nExc'], R.preEI, R.posEI)
aIE = adjacency_matrix_from_flat_inds(R.p['nExc'], R.p['nInh'], R.preIE, R.posIE)
aII = adjacency_matrix_from_flat_inds(R.p['nInh'], R.p['nInh'], R.preII, R.posII)

nIncomingExcOntoEachExc = aEE.sum(0)
nIncomingInhOntoEachExc = aEI.sum(0)
nIncomingExcOntoEachInh = aIE.sum(0)
nIncomingInhOntoEachInh = aII.sum(0)

t0 = datetime.now()

for trialInd in range(nTrials):
    
    movAvgUpFRExcUnits = R.trialUpFRExcUnits[trialInd, :] * Hz
    movAvgUpFRInhUnits = R.trialUpFRInhUnits[trialInd, :] * Hz
    
    movAvgUpFRInhUnitsPreToPostExc = np.matmul(movAvgUpFRInhUnits, aEI) / nIncomingInhOntoEachExc
    movAvgUpFRExcUnitsPreToPostInh = np.matmul(movAvgUpFRExcUnits, aIE) / nIncomingExcOntoEachInh

    movAvgUpFRExcUnits[movAvgUpFRExcUnits < 1 * Hz] = 1 * Hz
    movAvgUpFRInhUnits[movAvgUpFRInhUnits < 1 * Hz] = 1 * Hz

    movAvgUpFRInhUnitsPreToPostExc[movAvgUpFRInhUnitsPreToPostExc > (2 * R.p['setUpFRInh'] - 1 * Hz)] = (2 * R.p[
        'setUpFRInh'] - 1 * Hz)
    movAvgUpFRExcUnitsPreToPostInh[movAvgUpFRExcUnitsPreToPostInh > (2 * R.p['setUpFRExc'] - 1 * Hz)] = (2 * R.p[
        'setUpFRExc'] - 1 * Hz)

    movAvgUpFRExcUnits[movAvgUpFRExcUnits > (2 * R.p['setUpFRExc'] - 1 * Hz)] = (2 * R.p['setUpFRExc'] - 1 * Hz)
    movAvgUpFRInhUnits[movAvgUpFRInhUnits > (2 * R.p['setUpFRInh'] - 1 * Hz)] = (2 * R.p['setUpFRInh'] - 1 * Hz)
    
    if R.p['useRule'] == 'cross-homeo-pre-outer':
        
        dwEE = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                      (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwEI = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                       (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwIE = -R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                       (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz
        dwII = R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                      (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))

    elif R.p['useRule'] == 'cross-homeo-pre-scalar':

        dwEECH1 = R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwEICH1 = -R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwIECH1 = -R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz
        dwIICH1 = R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz

        dwEE = np.tile(dwEECH1.reshape(-1, 1), R.p['nExc'])
        dwEI = np.tile(dwEICH1.reshape(-1, 1), R.p['nExc'])
        dwIE = np.tile(dwIECH1.reshape(-1, 1), R.p['nInh'])
        dwII = np.tile(dwIICH1.reshape(-1, 1), R.p['nInh'])

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))

    elif R.p['useRule'] == 'cross-homeo-pre-outer-homeo':

        # cross-homeo with presynaptic multiplier (outer product)
        # since outer strips units and because of alpha we multiply by Hz to convert to amps
        dwEECH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                        (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwEICH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                         (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwIECH = -R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                         (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz
        dwIICH = R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                        (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz

        # regular homeo (outer product)
        # since outer strips units and because of alpha we multiply by Hz to convert to amps
        dwEEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwEIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwIEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz
        dwIIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz

        dwEE = dwEECH + dwEEH
        dwEI = dwEICH + dwEIH
        dwIE = dwIECH + dwIEH
        dwII = dwIICH + dwIIH

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))
        
    elif R.p['useRule'] == 'cross-homeo-pre-scalar-homeo':

        dwEECH = R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwEICH = -R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwIECH = -R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz
        dwIICH = R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz

        # regular homeo (outer product)
        # since outer strips units and because of alpha we multiply by Hz to convert to amps
        dwEEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwEIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwIEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz
        dwIIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz

        dwEE = dwEECH.reshape(-1, 1) + dwEEH
        dwEI = dwEICH.reshape(-1, 1) + dwEIH
        dwIE = dwIECH.reshape(-1, 1) + dwIEH
        dwII = dwIICH.reshape(-1, 1) + dwIIH

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))
        
t1 = datetime.now()
print('took', t1-t0)

took 0:15:31.156521


In [102]:
f, ax = plt.subplots()
ax.plot(trialMAdwEE)

In [5]:
# check something about each file...

for seedInd in range(9):
    targetFile1, targetFile2 = analysisDict[seedInd]
    R1 = Results()
    try:
        R1.init_from_file(targetFile1, targetPath)
        print(R1.p['tauUpFRTrials'])
    except FileNotFoundError:
        print('one of the files wasn''t found for some reason')
    R2 = Results()
    try:
        R2.init_from_file(targetFile2, targetPath)
        print(R2.p['tauUpFRTrials'])
    except FileNotFoundError:
        print('one of the files wasn''t found for some reason')

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [20]:
# script to create all the figures for the multiTraining sessions

for seedInd in range(9):
    targetFile1, targetFile2 = analysisDict[seedInd]
    R1 = Results()
    R1.init_from_file(targetFile1, targetPath)
    R2 = Results()
    R2.init_from_file(targetFile2, targetPath)
    R = combine_two_sessions(R1, R2)

    print(R.rID)
    print(R.p['setUpFRExc'], R.p['tauUpFRTrials'], R.p['alpha1'], R.p['spikeInputAmplitude'])
    plt.close('all')
    pdfObject = PdfPages(targetPath + 'reports/' + R.rID + '_report.pdf')
    removeOutliers = False

    generate_description(R)

    FR_weights(R)
    plt.savefig(pdfObject, format='pdf')

    # firstConvergentInd = int(R.trialwEE.size / 2)
    # firstConvergentInd = 4700
    firstConvergentInd = determine_first_convergent_trial(R)
    # firstConvergentInd = determine_first_convergent_trial(R) + 1500
    print('We think the first convergent index is', firstConvergentInd)

    determine_drift_rates(R, firstConvergentInd)
    print(R.driftRateEE)
    print(R.driftRateEI)
    print(R.driftRateIE)
    print(R.driftRateII)

    # convergence process
    FR_scatter(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    FR_hist1d_compare(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    FR_weights_std(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    weights_scatter_matrix(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')

    # post convergence
    FR_scatter(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
    plt.savefig(pdfObject, format='pdf')
    FR_hist2d(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
    plt.savefig(pdfObject, format='pdf')
    FR_hist1d_compare(R, startTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    FR_weights_std(R, startTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    weights_scatter_matrix(R, startTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')

    # epilogue
    weights_hist1d_compare(R)
    plt.savefig(pdfObject, format='pdf')
    weights_matrix_compare(R)
    plt.savefig(pdfObject, format='pdf')

    pdfObject.close()

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed0__2021-08-21-09-33
5. Hz 1 8. fC 0.98
We think the first convergent index is 1164
0.3438049879554385
-0.25267338753683105
0.1026854251972901
-0.7582299550928737


WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in greater
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in less
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\analysis.py:320: RuntimeWarning: invalid value encountered in less
  wFullInitPlot[wFullInitPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:321: RuntimeWarning: invalid value encountered in greater
  wFullInitPlot[wFullInitPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:323: RuntimeWarning: invalid value encountered in less
  wFullFinalPlot[wFullFinalPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:324: RuntimeWarning: invalid value encountered in greater
  wFullFinalPlot[wFullFinalPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    C:\Users\mikejseay

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed1__2021-08-21-09-34
5. Hz 1 8. fC 0.98
We think the first convergent index is 536
1.9519028104149363
-0.05276630252854969
1.3321280892914467
-2.0132216651786075


WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in greater
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in less
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\analysis.py:320: RuntimeWarning: invalid value encountered in less
  wFullInitPlot[wFullInitPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:321: RuntimeWarning: invalid value encountered in greater
  wFullInitPlot[wFullInitPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:323: RuntimeWarning: invalid value encountered in less
  wFullFinalPlot[wFullFinalPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:324: RuntimeWarning: invalid value encountered in greater
  wFullFinalPlot[wFullFinalPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    C:\Users\mikejseay

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed2__2021-08-21-09-34
5. Hz 1 8. fC 0.98
We think the first convergent index is 5626
1.7345976444595907
0.7466571490226588
1.018813379420174
-1.0040977452787898


KeyboardInterrupt: 

In [34]:
overallDF.columns = overallDF.sort_index(axis=1, level=[0, 1], ascending=[True, True]).columns

In [30]:
plt.close('all')